# Clean sleep data

In [1]:
import pandas as pd
from zipfile import ZipFile

save_to_db = True

In [2]:
with ZipFile('../data/sleep-export.zip') as my_zip:
    with my_zip.open('sleep-export.csv', 'r') as f:
        lines = list(map(lambda x: x.decode('utf-8'), f.readlines()))

The file format is a modification of a csv file created by [Sleep as Android](https://docs.sleep.urbandroid.org/devs/csv.html). It needs to be parsed. There are three types of data to extract.

- Generic information for each sleep
- All [events](https://docs.sleep.urbandroid.org/devs/sleepcloud_api.html#event-labels) during a sleeping period
- Accelerometer information during a sleeping period

Initially we create two dataframes, one including the generic information and the other one the event and accelerometer information. Then, we split the latter into the event and accelerometer dataframes.

In [3]:
def get_sleeps(data: list[str]) -> list[list[str]]:
    while len(data) > 0:
        i = 2 if data[2].startswith('Id') else 3

        sleep, data = data[:i], data[i:]
        yield [list(map(lambda y: y.replace('"', '')
                        .replace('\n', ''), x.split(','))) for x in sleep]

In [4]:
sleep_data = []
event_data = []
for sleep in get_sleeps(lines):
    sleep_data.append(sleep[1][:15])

    for time, mov_value in zip(sleep[0][15:], sleep[1][15:]):
        event_data.append([sleep[1][0], time, mov_value])


headers = sleep[0][:15]

sleep_df = pd.DataFrame(sleep_data, columns=headers)
event_df = pd.DataFrame(event_data, columns=['Id', 'Time', 'Value'])
movement_df = event_df[event_df['Time'] != 'Event']
event_df = event_df[event_df['Time'] == 'Event'][['Id', 'Value']].rename(
    columns = {'Value': 'Event'}
)

Clean sleep data

In [5]:
for col in ('From', 'To', 'Sched'):
    sleep_df[col] = pd.to_datetime(sleep_df[col], format='%d. %m. %Y %H:%M')

for col in ('Id', 'Hours', 'Rating', 'DeepSleep', 'Framerate', 'Snore', 'Noise',
            'Cycles', 'LenAdjust'):
    sleep_df[col] = pd.to_numeric(sleep_df[col])

sleep_df = sleep_df.rename(
    columns={old: old.lower() for old in sleep_df.columns}
)
sleep_df = sleep_df.rename(columns={'from': 'sleep_from', 'to': 'sleep_to'})

In [6]:
# Parse tags in comments
sleep_df['tags'] = sleep_df["comment"].str.findall(r"#([a-z]*)").str.join(',')

,id,tz,sleep_from,sleep_to,sched,hours,rating,comment,framerate,snore,noise,cycles,deepsleep,lenadjust,geo,tags
0,1686529463485,Europe/Amsterdam,2023-06-12 02:24:00,2023-06-12 07:51:00,2023-06-12 07:40:00,5.45,0.0,#watch #geo1,10008,60,0.041469,4,0.540000,-24,e08d6816,"watch,geo"
1,1686440836740,Europe/Amsterdam,2023-06-11 01:47:00,2023-06-11 09:54:00,2023-06-11 10:40:00,8.12,0.0,#watch #geo1,10008,2430,0.073145,7,0.346939,-62,e08d6816,"watch,geo"
2,1686358567581,Europe/Amsterdam,2023-06-10 02:56:00,2023-06-10 09:47:00,2023-06-10 10:40:00,6.86,0.0,#watch #geo1,10008,90,0.025122,6,0.487179,0,e08d6816,"watch,geo"
3,1686259754450,Europe/Amsterdam,2023-06-08 23:29:00,2023-06-09 08:39:00,2023-06-09 08:40:00,9.16,0.0,#watch #geo1,10008,870,0.049774,9,0.400000,-2,e08d6816,"watch,geo"
4,1686179498318,Europe/Amsterdam,2023-06-08 01:11:00,2023-06-08 07:31:00,2023-06-08 07:40:00,6.33,0.0,#watch #auto #geo1,10008,1110,0.035014,4,0.302598,-17,e08d6816,"watch,auto,geo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,1562193748520,Europe/Madrid,2019-07-04 00:42:00,2019-07-04 07:42:00,2019-07-04 07:45:00,7.00,0.0,,10007,60,0.099169,8,0.576471,-5,,
1422,1562106785275,Europe/Madrid,2019-07-03 00:33:00,2019-07-03 07:16:00,2019-07-03 07:45:00,6.72,0.0,#newmoon,10007,60,0.071260,6,0.629630,-5,,newmoon
1423,1562018867201,Europe/Madrid,2019-07-02 00:07:00,2019-07-02 07:30:00,2019-07-02 07:45:00,7.38,0.0,,10007,0,0.143623,7,0.516854,-5,,
1424,1561934243995,Europe/Madrid,2019-07-01 00:37:00,2019-07-01 07:29:00,2019-07-01 07:45:00,6.87,0.0,#lullaby,10007,0,0.102219,8,0.578313,-5,,lullaby


Clean accelerometer data

In [7]:
for col in ('Id', 'Value'):
    movement_df[col] = pd.to_numeric(movement_df[col])

movement_df['Time'] = pd.to_datetime(movement_df['Time'], format='%H:%M').dt.time

movement_df = movement_df.rename(
    columns={old: old.lower() for old in movement_df.columns}
    )

event_df['Id'] = pd.to_numeric(event_df['Id'])

event_df = event_df.rename(
    columns={old: old.lower() for old in event_df.columns}
    )

Save clean data

In [8]:
sleep_df.to_csv('../data/clean/sleeps.csv', index=False)
movement_df.to_csv('../data/clean/movements.csv', index=False)
event_df.to_csv('../data/clean/events.csv', index=False)

There is more cleaning to be done. 

For instance, I only used the comment feature of the app once when I downloaded and it has an irrelevant comment in Catalan, but the app automatically adds different tags that can be extracted into a new table of its own.

The events dataframe includes all events that happen in a night with a timestamp. Again, this can be parsed and utilized.

## Put raw data in SQLite3 database

Since the idea is to use dbt for the data transformation until we get analysis ready tables, the data will also be added to a locally managed SQLite3 database.

In [9]:
import sqlite3

con = sqlite3.connect('../database/raw.db')

In [10]:
con.execute('DROP TABLE IF EXISTS sleeps;')
con.execute('DROP TABLE IF EXISTS movements;')
con.execute('DROP TABLE IF EXISTS events;')

In [11]:
sleep_df.to_sql('sleeps', con, index=False)
movement_df.to_sql('movements', con, index=False)
event_df.to_sql('events', con, index=False)

228669